In [37]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [38]:
import cv2
import os
from os import listdir
from sklearn.preprocessing import LabelBinarizer,MultiLabelBinarizer
from sklearn.model_selection import train_test_split

In [39]:
root_dir = r'C:\Users\javed\Downloads\archive (7)\PlantVillage'

os.chdir(root_dir)
listdir()

['train']

In [40]:
# Dimension of resized image
DEFAULT_IMAGE_SIZE = tuple((256, 256))

# Number of images used to train the model
N_IMAGES = 100

data_dir = os.path.join(root_dir,'train')

"""We use the function `convert_image_to_array` to resize an image to the size `DEFAULT_IMAGE_SIZE` we defined above."""

def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None:
            image = cv2.resize(image, DEFAULT_IMAGE_SIZE)   
            return img_to_array(image)
        else:
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None
     



In [41]:

print("Load images from all classes ...")
plant_disease_folder_list = listdir(data_dir)
print(len(plant_disease_folder_list))

Load images from all classes ...
15


In [42]:
image_list, label_list = [], []

try:
    print("Loding Image Data ...")
    for s in listdir():
        plant_disease_folder_list = listdir(s)

        for plant_disease_folder in plant_disease_folder_list:
            print(f"Processing  {plant_disease_folder} ...")
            plant_disease_image_list = listdir(f"{data_dir}/{plant_disease_folder}/")

            for image in plant_disease_image_list[:N_IMAGES]:
                image_directory = f"{data_dir}/{plant_disease_folder}/{image}"
                if image_directory.endswith(".jpg")==True or image_directory.endswith(".JPG")==True:
                    image_list.append(convert_image_to_array(image_directory))
                    label_list.append(plant_disease_folder)

    print("All the images have successfully loaded!!")  
except Exception as e:
    print(f"Error : {e}")

Loding Image Data ...
Processing  Pepper__bell___Bacterial_spot ...
Processing  Pepper__bell___healthy ...
Processing  Potato___Early_blight ...
Processing  Potato___healthy ...
Processing  Potato___Late_blight ...
Processing  Tomato_Bacterial_spot ...
Processing  Tomato_Early_blight ...
Processing  Tomato_healthy ...
Processing  Tomato_Late_blight ...
Processing  Tomato_Leaf_Mold ...
Processing  Tomato_Septoria_leaf_spot ...
Processing  Tomato_Spider_mites_Two_spotted_spider_mite ...
Processing  Tomato__Target_Spot ...
Processing  Tomato__Tomato_mosaic_virus ...
Processing  Tomato__Tomato_YellowLeaf__Curl_Virus ...
All the images have successfully loaded!!


In [43]:

print(len(image_list))

1500


In [44]:
# Transform the loaded training image data into numpy array
np_image_list = np.array(image_list, dtype=np.float16) / 255.0

# Check the number of images loaded for training
image_len = len(image_list)
print(f"Total number of images: {image_len}")

Total number of images: 1500


In [45]:
"""Examine the labels/classes in the training dataset."""

label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)

# pickle.dump(label_binarizer,open('plant_disease_label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)

print("Total number of classes: ", n_classes)

Total number of classes:  15


In [46]:

augment = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
                             height_shift_range=0.1, shear_range=0.2, 
                             zoom_range=0.2, horizontal_flip=True, 
                             fill_mode="nearest")

In [47]:
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state = 42)
print('Successfully split data into TRAIN & TEST')

Successfully split data into TRAIN & TEST


In [52]:
from tensorflow.keras.applications import VGG16, VGG19, ResNet50, InceptionV3, MobileNet

# VGG16

In [60]:
num_classes = 15

# Add custom top layers
model_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
for layer in model_vgg16.layers:
    layer.trainable = False


In [61]:

# Add custom top layers
x = model_vgg16.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x) # You can adjust the number of units as needed
x = Dropout(0.5)(x) # Optional dropout layer for regularization
predictions = Dense(num_classes, activation='softmax')(x)


In [62]:

# Create a new model with custom top layers
model_vgg16 = Model(inputs=model_vgg16.input, outputs=predictions)


In [63]:

# Compile the model
model_vgg16.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



In [64]:
# Train the model
model_vgg16.fit(x_train, y_train, epochs=5, batch_size=32, validation_data=(x_test, y_test))


Epoch 1/5
38/38 [==============================] - 280s 7s/step - loss: 5.3047 - accuracy: 0.2750 - val_loss: 1.2992 - val_accuracy: 0.6133
Epoch 2/5
38/38 [==============================] - 296s 8s/step - loss: 1.1537 - accuracy: 0.6275 - val_loss: 0.8770 - val_accuracy: 0.7733
Epoch 3/5
38/38 [==============================] - 287s 8s/step - loss: 0.8638 - accuracy: 0.7292 - val_loss: 0.7830 - val_accuracy: 0.7567
Epoch 4/5
38/38 [==============================] - 283s 7s/step - loss: 0.7085 - accuracy: 0.7750 - val_loss: 0.6776 - val_accuracy: 0.7900
Epoch 5/5
38/38 [==============================] - 289s 8s/step - loss: 0.5277 - accuracy: 0.8358 - val_loss: 0.6655 - val_accuracy: 0.7967


In [65]:
test_lossVGG16, test_accVGG16 = model_vgg16.evaluate(x_test, y_test)
print(f"Model: VGG16")
print(f"Test Accuracy: {test_accVGG16}")

10/10 [==============================] - 45s 4s/step - loss: 0.6655 - accuracy: 0.7967
Model: VGG16
Test Accuracy: 0.79666668176651


#  Model 2: VGG19

In [66]:
# Model 2: VGG19
model_vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=(256, 256, 3)) # Adjust input shape as needed
for layer in model_vgg19.layers:
    layer.trainable = False # Freeze layers



In [70]:
from tensorflow.keras.models import Sequential

# Load VGG19 base
vgg19_base = VGG19(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Create a Sequential model
model_vgg19 = Sequential()

# Add the VGG19 base
model_vgg19.add(vgg19_base)

# Add custom top layers
model_vgg19.add(Flatten())
model_vgg19.add(Dense(256))
model_vgg19.add(Activation('relu'))
model_vgg19.add(Dense(15))  # 15 units for 15 classes
model_vgg19.add(Activation('softmax'))  # Softmax activation for multi-class classification


In [71]:
# Compile the model
model_vgg19.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [72]:

# Train the model
model_vgg19.fit(x_train, y_train, epochs=5, batch_size=32, validation_data=(x_test, y_test))








Epoch 1/5
38/38 [==============================] - 1183s 31s/step - loss: 5.0912 - accuracy: 0.0608 - val_loss: 2.7083 - val_accuracy: 0.0500
Epoch 2/5
38/38 [==============================] - 1072s 28s/step - loss: 2.7083 - accuracy: 0.0667 - val_loss: 2.7088 - val_accuracy: 0.0500
Epoch 3/5
38/38 [==============================] - 1095s 29s/step - loss: 2.7083 - accuracy: 0.0625 - val_loss: 2.7096 - val_accuracy: 0.0500
Epoch 4/5
38/38 [==============================] - 1092s 29s/step - loss: 2.7080 - accuracy: 0.0708 - val_loss: 2.7099 - val_accuracy: 0.0500
Epoch 5/5
38/38 [==============================] - 1046s 28s/step - loss: 2.7080 - accuracy: 0.0708 - val_loss: 2.7103 - val_accuracy: 0.0500


In [73]:
test_lossVGG19, test_accVGG19 = model_vgg19.evaluate(x_test, y_test)
print(f"Model: VGG19")
print(f"Test Accuracy: {test_accVGG19}")
print("----------------------")

10/10 [==============================] - 108s 11s/step - loss: 2.7103 - accuracy: 0.0500
Model: VGG19
Test Accuracy: 0.05000000074505806
----------------------


# Model 3: MobileNet

In [87]:

model_mobilenet = MobileNet(weights='imagenet', include_top=False, input_shape=(256, 256, 3)) 

In [88]:
# Add custom top layers
model = Sequential([
    model_mobilenet,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(15, activation='softmax')  # Adjust to match the number of classes in your dataset
])


In [89]:

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [90]:

model.fit(x_train, y_train, epochs=5, batch_size=32, validation_data=(x_test, y_test))


Epoch 1/5
38/38 [==============================] - 230s 6s/step - loss: 1.3026 - accuracy: 0.6083 - val_loss: 8.3132 - val_accuracy: 0.1033
Epoch 2/5
38/38 [==============================] - 207s 5s/step - loss: 0.3301 - accuracy: 0.9050 - val_loss: 4.6731 - val_accuracy: 0.3367
Epoch 3/5
38/38 [==============================] - 208s 5s/step - loss: 0.2637 - accuracy: 0.9100 - val_loss: 2.4904 - val_accuracy: 0.5500
Epoch 4/5
38/38 [==============================] - 206s 5s/step - loss: 0.2245 - accuracy: 0.9333 - val_loss: 3.1870 - val_accuracy: 0.5133
Epoch 5/5
38/38 [==============================] - 206s 5s/step - loss: 0.1761 - accuracy: 0.9475 - val_loss: 5.1954 - val_accuracy: 0.3967


In [91]:
# Now, evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Model: MobileNet")
print(f"Test Accuracy: {test_acc}")
print("----------------------")

10/10 [==============================] - 10s 1s/step - loss: 5.1954 - accuracy: 0.3967
Model: MobileNet
Test Accuracy: 0.39666667580604553
----------------------


# Model 4: ResNet50

In [92]:
# Model 3: ResNet50
model_resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3)) # Adjust input shape as needed
for layer in model_resnet50.layers:
    layer.trainable = False # Freeze layers



94765736/94765736 [==============================] - 236s 2us/step


In [103]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

# Load the ResNet50 model without the top layer (include_top=False)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Create a new model on top of the ResNet50 backbone
model_resnet50_custom = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(15, activation='softmax')  # Assuming 15 classes
])


In [104]:

# Compile the model
model_resnet50_custom.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



In [105]:
# Now, train and evaluate the model
model_resnet50_custom.fit(x_train, y_train, epochs=5, batch_size=32, validation_data=(x_test, y_test))


Epoch 1/5
38/38 [==============================] - 739s 18s/step - loss: 1.6593 - accuracy: 0.5458 - val_loss: 3104.8162 - val_accuracy: 0.0500
Epoch 2/5
38/38 [==============================] - 880s 23s/step - loss: 0.6190 - accuracy: 0.7942 - val_loss: 48.3105 - val_accuracy: 0.0500
Epoch 3/5
38/38 [==============================] - 672s 18s/step - loss: 0.3018 - accuracy: 0.8983 - val_loss: 9.2160 - val_accuracy: 0.0633
Epoch 4/5
38/38 [==============================] - 569s 15s/step - loss: 0.1880 - accuracy: 0.9317 - val_loss: 33.1855 - val_accuracy: 0.0500
Epoch 5/5
38/38 [==============================] - 503s 13s/step - loss: 0.1271 - accuracy: 0.9575 - val_loss: 51.8762 - val_accuracy: 0.0500


In [106]:
# Evaluate the model
test_loss, test_acc = model_resnet50_custom.evaluate(x_test, y_test)
print(f"Model: Custom ResNet50") 
print(f"Test Accuracy: {test_acc}")
print("----------------------")


10/10 [==============================] - 22s 2s/step - loss: 51.8762 - accuracy: 0.0500
Model: Custom ResNet50
Test Accuracy: 0.05000000074505806
----------------------


# Model 5: InceptionV3

In [111]:
# Model 4: InceptionV3
model_inceptionv3 = InceptionV3(weights='imagenet', include_top=False, input_shape=(256, 256, 3)) # Adjust input shape as needed
for layer in model_inceptionv3.layers:
    layer.trainable = False # Freeze layers

In [112]:
# Assuming 'num_classes' is the number of classes in your dataset
num_classes = 15

# Add custom top layers
model5 = Sequential([
    model_inceptionv3,  # Pre-trained layers
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Final output layer with 'num_classes' units
])



In [113]:

model5.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [114]:
model5.fit(x_train, y_train, epochs=5, batch_size=32, validation_data=(x_test, y_test))



Epoch 1/5
38/38 [==============================] - 100s 2s/step - loss: 15.0715 - accuracy: 0.2742 - val_loss: 2.0058 - val_accuracy: 0.3833
Epoch 2/5
38/38 [==============================] - 87s 2s/step - loss: 1.8816 - accuracy: 0.3767 - val_loss: 1.4547 - val_accuracy: 0.5600
Epoch 3/5
38/38 [==============================] - 86s 2s/step - loss: 1.5835 - accuracy: 0.4900 - val_loss: 1.3267 - val_accuracy: 0.5600
Epoch 4/5
38/38 [==============================] - 83s 2s/step - loss: 1.3578 - accuracy: 0.5600 - val_loss: 1.2392 - val_accuracy: 0.6700
Epoch 5/5
38/38 [==============================] - 82s 2s/step - loss: 1.2593 - accuracy: 0.5758 - val_loss: 1.2164 - val_accuracy: 0.6133


In [115]:
test_loss, test_acc = model5.evaluate(x_test, y_test)
print(f"Model: InceptionV3")
print(f"Test Accuracy: {test_acc}")
print("----------------------")

10/10 [==============================] - 12s 1s/step - loss: 1.2164 - accuracy: 0.6133
Model: InceptionV3
Test Accuracy: 0.6133333444595337
----------------------


In [121]:
model_vgg16.save('plant_village_vgg16.h5')

C:\Users\javed\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [122]:
model5.save('plant_village_InceptionV3.h5')